In [55]:
using DataFrames
using CSV
import Base.Iterators: flatten, zip
import IterTools.subsets

# Define struct

In [56]:
struct Rule
    ant::Array{Int64,1}
    con::Array{Int64,1}
end

In [57]:
Rule([1], [2,3])

Rule([1], [2, 3])

# Data load

In [58]:
onigiri_data = CSV.read("onigiri.csv", delim=",")
onigiri_array = Matrix(onigiri_data)

5×5 Array{Int64,2}:
 0  1  1  1  0
 1  1  0  0  0
 1  0  0  1  1
 0  1  1  1  0
 0  1  0  1  0

In [59]:
function GTI(mat::Array{Int64,2}, indexes::Array{Int64,1})
    return mat[:,indexes] .==1
end

GTI (generic function with 1 method)

In [60]:
[2]

1-element Array{Int64,1}:
 2

In [61]:
GTI(onigiri_array,[2])

5×1 BitArray{2}:
 1
 1
 0
 1
 1

In [62]:
onigiri_array

5×5 Array{Int64,2}:
 0  1  1  1  0
 1  1  0  0  0
 1  0  0  1  1
 0  1  1  1  0
 0  1  0  1  0

In [63]:
function support(array_2d::Array{Int64,2}, indexes::Array{Int64,1}; m="num")
    gti_b = GTI(array_2d, indexes)
    if size(gti_b)[2] ==0
        return 0
    end
    b = all(gti_b, dims=2)
    if m =="num"
        return sum(b)
    elseif m == "ratio"
        return sum(b)/length(b)
    elseif m=="bool"
        return b
    end
end

            
    
        
            

support (generic function with 1 method)

In [64]:
function confidence(array_2d::Array{Int64,2}, 
        X_indexes::Array{Int64,1}, Y_indexes::Array{Int64,1})
    sup_X = support(array_2d, X_indexes)
    X_Y_indexes = cat(X_indexes, Y_indexes, dims=1)
    return support(array_2d, X_Y_indexes)/sup_X
end

confidence (generic function with 1 method)

In [65]:
function getF1(array_2D::Array{Int64,2}, minsup::Float64)
    return [[col] for col in 1:size(array_2D)[2] if support(array_2D, [col], m="ratio") >= minsup]
end

getF1 (generic function with 1 method)

In [66]:
getF1(onigiri_array, 0.4)

4-element Array{Array{Int64,1},1}:
 [1]
 [2]
 [3]
 [4]

In [67]:
function getFkPlusOne(array_2D::Array{Int64,2}, indexes::Array{Array{Int64,1},1}, minsup::Float64)
    return [col for col in indexes if support(array_2D, col, m="ratio") >= minsup]
                
end

getFkPlusOne (generic function with 1 method)

In [68]:
function getCkPlusOne(prevCandidate::Array{Array{Int64, 1}, 1}, k)
    @assert all(length.(prevCandidate) .==  k-1)
    @assert k>1
    items = unique(collect(flatten(prevCandidate)))
    tmp_candidates = [x for x in subsets(items, k)]
    if k ==2
        return tmp_candidates
    end
    
    candidates = [
        candidate for candidate in tmp_candidates
        if all(
                x in prevCandidate
                for x in subsets(candidate, k-1))
    ]
                
    return candidates
                
end

getCkPlusOne (generic function with 1 method)

In [69]:
function isEmpty(F::Array{Array{Int64,1},1})
    if length(F) < 1
        return true
    else
        return false
    end
end

isEmpty (generic function with 1 method)

In [70]:
function isCalcConfNeeded(array_prev_ant::Array{Array{Int64,1},1},
                    array_ant::Array{Array{Int64,1},1}, set_f::Array{Int64,1})
    array_prev_con = [setdiff(set_f,  set_c) for set_c in array_ant]
    array_con = [setdiff(set_f, set_c) for set_c in array_ant]
    
    out = []
    for (a,c) in zip(array_ant, array_con)
        out_inner = []
        for i in 1:length(c)

            array_ant_candidate = a
            cand = c[i]
            array_candidate_ant = vcat(a, cand)
            array_candidate_con = filter(x ->x != cand, c)
            
            res = any([issubset(array_candidate_ant, i) for i in array_prev_ant])
            append!(out_inner, res)
        end
        if all(out_inner)
            append!(out, true)
        else
            append!(out, false)
        end
    end
    
    out = convert(Array{Bool, 1}, out)

    return out
end

                
            

isCalcConfNeeded (generic function with 1 method)

In [162]:
function frequent(array_2D::Array{Int64,2}; minsum::Float64, colnum::Int64)
    k = 1
    F_now = getF1(array_2D, minsum)
    F_list = []
    F_table = zeros(1,colnum) # first line is dummy (all zero)
    append!(F_list, [F_now])
    
    F_table = cat(F_table, label_encode)
    
    
    while(true)
        C_next = getCkPlusOne(F_now, k+1)
        F_next = getFkPlusOne(array_2D, C_next, minsum)
        
        if isEmpty(F_next)
            break
        end
        k += 1
        F_now = F_next
        append!(F_list, [F_now])
    end
    
    F_list = convert(Array{Array{Array{Int64,1},1},1}, F_list)
    
    return F_list
end

frequent (generic function with 1 method)

In [107]:
_F_list = frequent(onigiri_array, minsum=0.4)

3-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4]]    
 [[2, 3], [2, 4], [3, 4]]
 [[2, 3, 4]]             

In [232]:
function itemlist2table(item_list::Array{Array{Array{Int64,1},1},1},
                                             col_num::Int64)
    table = []
    for k in item_list
        for item in k
            #label encode
            arr = zeros(Int64, 1,col_num)
            arr[1,item] .= 1
            
            table = cat(table, arr, dims=1)
        end
    end
    return table
end

            

itemlist2table (generic function with 5 methods)

In [231]:
tmp = itemlist2table( _F_list, 10)

8×10 Array{Any,2}:
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  1  1  0  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0
 0  1  1  1  0  0  0  0  0  0

In [197]:
function find_rules(array_2D::Array{Int64, 2}, 
        F_list::Array{Array{Array{Int64,1},1},1}; minconf::Float64)
    conf_list = []
    
    
    for F in F_list
        k = length(F[1])
        
        if k == 1
            #conf_list = vcat(conf_list, Any{[Rule([0],[0])]}) # DUMMY!!
        
        elseif k == 2
            conf_list_k = []
            for f_2 in F
                A = f_2[1]
                B = f_2[2]
                conf_AB = confidence(array_2D, [A], [B])
                if conf_AB >= minconf
                    #append!(conf_list_k, Rule([A],[B]))
                    conf_list_k = vcat(conf_list_k, Rule([A],[B]))

                end
                conf_BA = confidence(array_2D, [B], [A])
                if conf_BA >= minconf
                    #append!(conf_list_k, Rule([B],[A]))
                    conf_list_k = vcat(conf_list_k, Rule([B],[A]))
                end
            end
            append!(conf_list, [conf_list_k])   


        elseif k >= 3
            conf_list_k = []
            for f_k in F
                
                j = 1
                
                array_antecedent =  collect(subsets(f_k, k-1))
                array_consequent = [setdiff(f_k,  set_c) for set_c in array_antecedent]
                conf = [confidence(array_2D, ant, con) for (ant, con) in zip(array_antecedent, array_consequent)]
                isHigher = conf .>= minconf
                if sum(isHigher) > 0
                    array_antecedent_filtered_by_conf = array_antecedent[isHigher]
                    array_consequent_filtered_by_conf = array_consequent[isHigher]
                    append!(conf_list_k, [Rule(a,c) for (a,c) in zip(array_antecedent_filtered_by_conf,
                                                                                            array_consequent_filtered_by_conf)])
                    
                    while(j < k-1)
                        array_antecedent_new = collect(subsets(f_k, k-(j+1)))
                        _res = isCalcConfNeeded(array_antecedent_filtered_by_conf, array_antecedent_new, f_k)
                        if sum(_res) > 0
                            array_antecedent_filtered_by_prev = array_antecedent_new[_res]
                            array_consequent_filtered_by_prev = [setdiff(f_k,  set_c) 
                                                                                            for set_c in array_antecedent_filtered_by_prev]
                            conf = [confidence(array_2D, ant, con) for (ant, con) in zip(array_antecedent_filtered_by_prev, 
                                                                                                                                array_consequent_filtered_by_prev)]
                            isHigher = conf .>= minconf
                            if sum(isHigher) > 0
                                array_antecedent_filtered_by_prev_and_conf = array_antecedent_filtered_by_prev[isHigher]
                                array_consequent_filtered_by_prev_and_conf = array_consequent_filtered_by_prev[isHigher]
                                append!(conf_list_k, [Rule(a,c) for (a,c) in zip(array_antecedent_filtered_by_prev_and_conf, 
                                                                                                         array_consequent_filtered_by_prev_and_conf)])
                            end
                        end
                        j += 1
                    end #while
                end
            end
            append!(conf_list, [conf_list_k])
        end
    end
    conf_list = convert(Array{Array{Rule,1},1}, conf_list)
    return conf_list
end

find_rules (generic function with 1 method)

In [74]:
find_rules(onigiri_array, _F_list, minconf=0.7)

2-element Array{Array{Rule,1},1}:
 [Rule([3], [2]), Rule([2], [4]), Rule([4], [2]), Rule([3], [4])]
 [Rule([2, 3], [4]), Rule([3, 4], [2]), Rule([3], [2, 4])]       

In [ ]:
store_data = CSV.read("store_data_trans.csv", delim=",")
store_array = Matrix(store_data)
store_array

In [ ]:
minsum = 0.007
F_list = frequent(store_array, minsum)

In [ ]:
minconf = 0.01
find_rules(store_array, F_list, minconf)

# mnist

In [234]:
mnist_data = CSV.read("mnist_8x8_image.csv", delim=",",  header=false)
mnist_array = Matrix(mnist_data)
mnist_array = convert(Array{Int64, 2}, mnist_array)

mnist_label = CSV.read("mnist_8x8_label.csv", delim=",",  header=false)
mnist_label = Matrix(mnist_label)
mnist_label = convert(Array{Int64, 2}, mnist_label)

mnist_label_onehot = CSV.read("mnist_8x8_label_onehot.csv", delim=",",  header=false)
mnist_label_onehot = Matrix(mnist_label_onehot)
mnist_label_onehot = convert(Array{Int64, 2}, mnist_label_onehot)


898×10 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 ⋮              ⋮            
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0
 0  0  1  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  1  0  0  0  0
 0  0  0  0  1  0  0  0  0  0

In [254]:
SubArray(mnist_array, mnist_label .==3)

MethodError: MethodError: no method matching SubArray(::Array{Int64,2}, ::BitArray{2})
Closest candidates are:
  SubArray(::AbstractArray, !Matched::Tuple) at subarray.jl:21
  SubArray(!Matched::IndexCartesian, ::P, !Matched::I, !Matched::Tuple{Vararg{Any,N}}) where {P, I, N} at subarray.jl:25
  SubArray(!Matched::IndexLinear, ::P, !Matched::I, !Matched::Tuple{Vararg{Any,N}}) where {P, I, N} at subarray.jl:29

## concat image and label

In [93]:
mnist_cat = cat(mnist_array, mnist_label, dims=2)

898×74 Array{Int64,2}:
 0  0  0  1  1  0  0  0  0  0  1  1  1  …  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  1  1  0  0  0  0  0  0  1  1     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  1  1     1  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  1  1  0  0  0  0  1  1  0  1     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  1     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0  1  1  1  …  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  1  1  0  0  0  0  0  0  1  1     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  1  1  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  1  0  0
 0  0  1  1  1  0  0  0  0  0  1  1  1     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  1  1  0  0  0  0  0  0  1  1  1     0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  1  1  1  0  0  0  0  1  1  1  …  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  0  1     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  1  1  0   

# Julia

In [79]:
mnist_data

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
9,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [220]:
@time F_list = frequent(mnist_array, minsum=0.3)

  1.032128 seconds (3.12 M allocations: 321.115 MiB, 5.13% gc time)


7-element Array{Array{Array{Int64,1},1},1}:
 [[4], [5], [6], [11], [12], [13], [14], [19], [20], [21]  …  [44], [45], [46], [51], [52], [53], [54], [60], [61], [62]]                                                                                                                                                                                                                                                                                                                                                                                          
 [[4, 5], [4, 11], [4, 12], [4, 13], [4, 14], [4, 19], [4, 20], [4, 21], [4, 22], [4, 27]  …  [52, 62], [53, 54], [53, 60], [53, 61], [54, 60], [54, 61], [54, 62], [60, 61], [60, 62], [61, 62]]                                                                                                                                                                                                                                                                       

In [221]:
F_list[3]

1015-element Array{Array{Int64,1},1}:
 [4, 5, 11]  
 [4, 5, 12]  
 [4, 5, 13]  
 [4, 5, 14]  
 [4, 5, 19]  
 [4, 5, 22]  
 [4, 5, 27]  
 [4, 5, 28]  
 [4, 5, 29]  
 [4, 5, 30]  
 [4, 5, 35]  
 [4, 5, 36]  
 [4, 5, 37]  
 ⋮           
 [52, 53, 61]
 [52, 54, 60]
 [52, 54, 61]
 [52, 60, 61]
 [52, 61, 62]
 [53, 54, 60]
 [53, 54, 61]
 [53, 60, 61]
 [54, 60, 61]
 [54, 60, 62]
 [54, 61, 62]
 [60, 61, 62]

In [219]:
F_table = itemlist2table(F_list, size(mnist_cat)[2])

3256×74 Array{Any,2}:
 0  0  0  1  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0    

In [206]:
size(mnist_cat)[2]

74

In [99]:
@time F_list = frequent(mnist_cat, minsum=0.15)

 16.950267 seconds (34.17 M allocations: 2.117 GiB, 7.33% gc time)


7-element Array{Array{Array{Int64,1},1},1}:
 [[3], [4], [5], [6], [11], [12], [13], [14], [19], [20]  …  [47], [51], [52], [53], [54], [55], [59], [60], [61], [62]]                                                                                                                                                                                                                                                                                                                                                                                               
 [[3, 4], [3, 5], [3, 11], [3, 12], [3, 13], [3, 28], [3, 29], [3, 36], [3, 37], [3, 52]  …  [54, 61], [54, 62], [55, 60], [55, 61], [55, 62], [59, 60], [59, 61], [60, 61], [60, 62], [61, 62]]                                                                                                                                                                                                                                                                    

In [106]:
@time F_list = frequent(mnist_cat, minsum=0.09)

InterruptException: InterruptException:

In [105]:
@time F_list = frequent(mnist_label, minsum=0.09)

  0.000895 seconds (1.08 k allocations: 1005.000 KiB)


1-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]]

In [104]:
@time find_rules(mnist_array, F_list, minconf=0.1)

  0.000012 seconds (12 allocations: 544 bytes)


0-element Array{Array{Rule,1},1}

In [51]:
mnist_array

897×64 Array{Int64,2}:
 0  0  0  1  1  0  0  0  0  0  0  1  1  …  1  0  0  0  0  0  0  1  1  1  0  0
 0  0  0  0  1  1  0  0  0  0  0  1  1     1  1  0  0  0  0  0  0  1  1  1  0
 0  0  0  1  1  0  0  0  0  1  1  0  1     0  1  1  0  0  0  0  1  1  1  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  1     1  1  0  0  0  0  0  0  1  0  0  0
 0  0  1  1  0  0  0  0  0  0  1  1  1     1  1  0  0  0  0  1  1  1  1  0  0
 0  0  0  1  1  0  0  0  0  0  0  1  1  …  1  1  1  0  0  0  0  1  1  1  0  0
 0  0  0  1  1  1  1  0  0  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  1  1  1  0  0  0  0  0  1  1  1     0  1  1  0  0  0  1  1  1  1  0  0
 0  0  1  1  0  0  0  0  0  0  1  1  1     1  1  0  0  0  0  1  1  1  0  0  0
 0  0  0  1  1  1  0  0  0  0  1  1  1     1  1  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  0  1  …  1  1  0  0  0  0  0  0  1  1  0  0
 0  0  0  1  0  0  0  0  0  0  1  1  0     1  1  1  0  0  0  0  1  1  1  1  0
 0  0  1  1  1  1  0  0  0  0  1  1  1   